In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import BernoulliNB

import pandas as pd

import numpy as np
import seaborn as sns
 
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib
import json
import os
plt.style.use('ggplot')
from matplotlib.pyplot import figure
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


matplotlib.rcParams['figure.figsize'] = (12,8)
 
pd.options.mode.chained_assignment = None

#Leitura dos dados
train = pd.read_json("/kaggle/input/whats-cooking-kernels-only/train.json")
test = pd.read_json("/kaggle/input/whats-cooking-kernels-only/test.json")
# print("Tipagem dos dados")
# print(train.dtypes)
 
# print("\nVolume de dados (linhas x colunas)")
# print(train.shape)
 
# print("\nColunas numericas")
df_numeric = train.select_dtypes(include=[np.number])
numeric_cols = df_numeric.columns.values
# print(numeric_cols)
 
 
# print("\nColunas não numericas (STRING)")
df_non_numeric = train.select_dtypes(exclude=[np.number])
non_numeric_cols = df_non_numeric.columns.values
print(non_numeric_cols)
 
colours = ['#8B0000', '#000000'] # Definição das cores para a plotagem (vermelho: dado presente, preto: dado não presente)
aux = sns.heatmap(train[train.columns].isnull(), cmap=sns.color_palette(colours))

train['ingredient_text'] = train['ingredients'].apply(lambda x: ' '.join([i.replace(' ','_').lower() for i in x]))
test['ingredient_text'] = test['ingredients'].apply(lambda x: ' '.join([i.replace(' ','_').lower() for i in x]))

def generate_parameters(x,y,classifier,parameters): 
  grid = GridSearchCV(
          classifier(), parameters, scoring='accuracy'
      )
  grid = grid.fit(x, y)
  clf = grid.best_estimator_
  return clf

def run_kfold(clf, train, test):
    print("rodando kfold")
    kf = KFold(10)
    print(kf)
    output = []
    fold = 0
    for train_index, test_index in kf.split(train):
        fold += 1
        x_train, x_test = train[train_index], train[test_index]
        y_train, y_test = test[train_index], test[test_index]
        clf.fit(x_train, y_train)
        predictions = clf.predict(x_test)
        accuracy = accuracy_score(y_test, predictions)
        output.append(accuracy)
        print("Fold {0} acurácia: {1}".format(fold, accuracy))     
    mean_output = np.mean(output)
    dp = np.std(output)
    print("Acurácia média: {0}".format(mean_output)) 
    print("Desvio padrão da acurácia: {0}".format(dp))

# x_all = train.drop(['cuisine'], axis=1)
# y_all = train['cuisine']

num_test = 0.2
cv = CountVectorizer()
le = LabelEncoder()

x_train = cv.fit_transform(train['ingredient_text'])
y_train = le.fit_transform(train['cuisine'])
x_test = cv.transform(test['ingredient_text'])
print(x_train)

nb = BernoulliNB()
parameters =[{
    'alpha': [1.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9],
    'binarize': [0.0,None, 0.5],
    'fit_prior': [True, False],    
}]
# print("Naive Bayes")
clf_nb = generate_parameters(x_train, y_train, BernoulliNB, parameters)

clf_nb.fit(x_train, y_train)

p = clf_nb.predict(x_test)
p = le.inverse_transform(p)
result = pd.DataFrame({'id': test["id"], 'cuisine': p})
result.to_csv("./submission.csv", index=False)
print(result)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session